In [6]:
import os,sys
sys.path.append('../src/')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [7]:
datapath_in = '/home/pedro/Downloads/logradouros'
datapath_out = '../data/interim/'

In [8]:
# Loading all dbf files into one single dataframe:
df_list = []

for obj in os.listdir(datapath_in):
    logradouros_path = os.path.join(datapath_in, obj)
    if os.path.isdir(logradouros_path):
        uf = obj.split('_')[0]
        print(f"Loading files for {logradouros_path} folder ({uf})...")
        for filename in os.listdir(logradouros_path):
            if filename[-4:] == '.dbf':                
                filepath = os.path.join(logradouros_path, filename)
                # print(filepath)

                dbf = Dbf5(filepath)
                df = dbf.to_dataframe()
                df['uf'] = uf
                df_list.append(df)
print("Done!")
df_all = pd.concat(df_list)

Loading files for /home/pedro/Downloads/logradouros/pr_faces_de_logradouros_2021 folder (pr)...
Loading files for /home/pedro/Downloads/logradouros/am_faces_de_logradouros_2021 folder (am)...
Loading files for /home/pedro/Downloads/logradouros/ce_faces_de_logradouros_2021 folder (ce)...
Loading files for /home/pedro/Downloads/logradouros/go_faces_de_logradouros_2021 folder (go)...
Loading files for /home/pedro/Downloads/logradouros/ac_faces_de_logradouros_2021 folder (ac)...
Loading files for /home/pedro/Downloads/logradouros/mt_faces_de_logradouros_2021 folder (mt)...
Loading files for /home/pedro/Downloads/logradouros/mg_faces_de_logradouros_2021 folder (mg)...
Loading files for /home/pedro/Downloads/logradouros/ma_faces_de_logradouros_2021 folder (ma)...
Loading files for /home/pedro/Downloads/logradouros/rj_faces_de_logradouros_2021 folder (rj)...
Loading files for /home/pedro/Downloads/logradouros/ap_faces_de_logradouros_2021 folder (ap)...
Loading files for /home/pedro/Downloads/

In [9]:
df_all

,CD_SETOR,CD_QUADRA,CD_FACE,NM_TIP_LOG,NM_TIT_LOG,NM_LOG,TOT_RES,TOT_GERAL,uf
0,411780005000001,001,001,RUA,DOUTOR,JOAO FERREIRA NEVES,5.0,7.0,pr
1,411780005000001,001,002,RUA,NaN,PITANGA,2.0,4.0,pr
2,411780005000001,001,003,RUA,ESCRIVA,EGLECI T G CAMPANINI,2.0,2.0,pr
3,411780005000001,001,004,RUA,IMACULADA,CONCEICAO,4.0,4.0,pr
4,411780005000001,002,001,RUA,DOUTOR,JOAO FERREIRA NEVES,2.0,3.0,pr
...,...,...,...,...,...,...,...,...,...
31475,110020545000017,030,004,AVENIDA,NaN,TANCREDO NEVES,0.0,2.0,ro
31476,110020545000017,031,001,AVENIDA,NaN,JUSCELINO KUBISTCHEK,0.0,0.0,ro
31477,110020545000017,031,002,RUA,NaN,IVO CARNEIRO,10.0,10.0,ro
31478,110020545000017,031,003,RUA,NaN,JOAO LEANDRO BARBOSA,1.0,1.0,ro


In [ ]:
# Saving file to output folder:
df_all.to_csv(os.path.join(datapath_out, 'logradouros.csv'), index=False)
df_all.to_parquet(os.path.join(datapath_out, 'logradouros.parquet.gzip'), compression='gzip')

In [1]:
import requests
import csv

def get_cep_from_address(address):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "format": "json",
        "q": address,
        "addressdetails": 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if data:
        address_details = data[0].get("address", {})
        return address_details.get("postcode")
    else:
        return None

input_csv_file = "/home/pedro/projects/hackaton-correios-cpbr15/data/interim/logradouros.csv"

output_generic_cep_csv_file = "/home/pedro/projects/hackaton-correios-cpbr15/data/interim/cep_generico.csv"
output_logradouro_cep_csv_file = "/home/pedro/projects/hackaton-correios-cpbr15/data/interim/cep_logradouro.csv"

with open(output_generic_cep_csv_file, "w", newline="") as generic_cep_file, \
     open(output_logradouro_cep_csv_file, "w", newline="") as logradouro_cep_file:

    # Definir os cabeçalhos dos arquivos de saída
    generic_cep_fieldnames = ["CD_SETOR", "CD_QUADRA", "CD_FACE", "NM_TIP_LOG", "NM_TIT_LOG", "NM_LOG", "TOT_RES", "TOT_GERAL", "uf", "CEP"]
    logradouro_cep_fieldnames = generic_cep_fieldnames 

    generic_writer = csv.DictWriter(generic_cep_file, fieldnames=generic_cep_fieldnames)
    logradouro_writer = csv.DictWriter(logradouro_cep_file, fieldnames=logradouro_cep_fieldnames)

    generic_writer.writeheader()
    logradouro_writer.writeheader()

    # Ler o arquivo CSV de entrada
    with open(input_csv_file, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            logradouro = row["NM_LOG"]
            estado = row["uf"]
            endereco = f"{logradouro}, {estado}"

            cep = get_cep_from_address(endereco)
            print(f"Processing the data ...\nCEP: {cep}\nEndereço: {endereco}")
            last_three_digits = cep[-3:]  if cep  else None
            row["CEP"] = cep
            if last_three_digits == "000" or last_three_digits == None:
                generic_writer.writerow(row)
            else:
                logradouro_writer.writerow(row)



Processing the data ...
CEP: None
Endereço: JOAO FERREIRA NEVES, pr
Processing the data ...
CEP: 85200-000
Endereço: PITANGA, pr
Processing the data ...
CEP: None
Endereço: EGLECI T G CAMPANINI, pr
Processing the data ...
CEP: None
Endereço: CONCEICAO, pr
Processing the data ...
CEP: None
Endereço: JOAO FERREIRA NEVES, pr
Processing the data ...
CEP: 80420-080
Endereço: VICENTE MACHADO, pr
Processing the data ...
CEP: None
Endereço: EGLECI T G CAMPANINI, pr
Processing the data ...
CEP: 85200-000
Endereço: PITANGA, pr
Processing the data ...
CEP: None
Endereço: JOAO FERREIRA NEVES, pr
Processing the data ...
CEP: 05000
Endereço: ISABEL, pr
Processing the data ...
CEP: None
Endereço: EGLECI T G CAMPANINI, pr
Processing the data ...
CEP: 80420-080
Endereço: VICENTE MACHADO, pr
Processing the data ...
CEP: None
Endereço: JOAO FERREIRA NEVES, pr
Processing the data ...
CEP: 20021-340
Endereço: SANTOS DUMONT, pr
Processing the data ...
CEP: None
Endereço: EGLECI T G CAMPANINI, pr
Processing 

ConnectionError: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?format=json&q=%2C+pr&addressdetails=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8dfcc244c0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))